In [3]:
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
from DISTS_pytorch import DISTS
from PIL import Image

import os

from pandas import *
import numpy as np
from scipy.io import loadmat, savemat

In [4]:
mask_ = loadmat('maskMat_isolated.mat')
#print(mask_)
mask = mask_['maskMat_isolated']

In [5]:

D = DISTS()
convert_tensor = transforms.ToTensor()

In [20]:
def cut_rectangle(prod,region):
    minx = prod.shape[2]
    miny = prod.shape[3]
    maxx = 0
    maxy = 0

    #prod has shape ([1, 3, 512, 512])
    #and you want to cut in the 512x512 bit.



    for k in range(region.shape[0]):
        for j in range(region.shape[1]):
            if prod[0,0,k,j]>0 or prod[0,1,k,j]>0 or prod[0,2,k,j]>0:
                if  minx>k:
                    minx = k
                elif maxx<k:
                    maxx = k
                if miny>j:
                    miny = j
                elif maxy<j:
                    maxy = j


    regx = prod[:,:,minx:maxx,miny:maxy]
    return regx,minx,maxx,miny,maxy


In [12]:
X = Image.open("datasave/original_images/fleamarket.jpg") #freeman metamer as reference
X = convert_tensor(X)
X = X[None,:]

Y = Image.open("datasave/img0_2/small_before_img_4_localloops.png") #freeman metamer as reference
Y = convert_tensor(Y)
Y = Y[None,:]

In [22]:
#code-wise, can I make a local DISTS model?
#probably yes. But I don't want to mask the DISTS network because 
#I don't know where I would do that. So I could use the segmenting code I have from previous iteration
#or just the idea from it anyway and do a quick test run, see if it's what Arturo wants :P
x_cutted = []
y_cutted = []

DISTS_final = 0
for region in mask: 
    #here, the shape of X is torch.Size([1, 3, 512, 512])
    region = torch.tensor(region)
    mask_stacked = region.repeat(3,1,1)
    
    temp_x = torch.zeros(X.shape)
    temp_x[0] = torch.mul(X[0,:,:,:],mask_stacked)
   
    temp_y = torch.zeros(Y.shape)
    temp_y[0] = torch.mul(Y[0,:,:,:],mask_stacked)

    regy,minx,maxx,miny,maxy = cut_rectangle(temp_x,region)

    X_cut =  X[:,:,minx:maxx,miny:maxy]
    Y_cut =  Y[:,:,minx:maxx,miny:maxy]

    #ok now given that this works, how we gonna compare?
    #compute dists scores for each portions of the image. Then scale and sum


    dists_value = D(X_cut,Y_cut)
    DISTS_final += dists_value * (np.count_nonzero(region)/(512*512))





torch.Size([1, 3, 512, 512]) torch.Size([3, 512, 512])
torch.Size([1, 3, 512, 512]) torch.Size([3, 512, 512])


KeyboardInterrupt: 